In [ ]:
#Please setup the api key below before excuting the code below.
dashscope.api_key = 'API-KEY'

In [ ]:
import random
from http import HTTPStatus
from dashscope import Generation
import dashscope
from flask import Flask, jsonify, request

#Setup the environment variable for sending the request to LLM (通义千问2-开源版-7B) 
dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'
dashscope.api_key = 'sk-4131fdc77e5a4abdb34e08c6942750fc'

#Define geners for movie classification
class_list = movie_genres = ["Action","Adventure","Comedy","Drama","Fantasy","Horror","Science Fiction","Romance","Thriller",
                            "Mystery","Historical","Music","Animation","Western","Documentary","Family","War","Crime"]
    

#Prepare 3 samples for model training
movies = [
    {
        "description": "A skilled thief, who specializes in stealing secrets from within the subconscious during the dream state, is given a chance to have his criminal history erased if he can successfully perform inception: planting an idea into a target's mind.",
        "genres": ["Action", "Sci-Fi", "Thriller"]
    },
    {
        "description": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",
        "genres": ["Drama", "Crime"]
    },
    {
        "description": "A writer encounters the former concierge of the Grand Budapest Hotel, who tells him the story of his life and the theft of a priceless Renaissance painting.",
        "genres": ["Comedy", "Drama", "Adventure"]
    }
]

#LLM based Movie Classifier
def movie_classifier(input_description):
    #Define the task required the model to do 
    messages = [{'role': 'system', 'content': 'You are a movie classifier, you are required to classify the movie into genre(s) based on the movie description user provided, you are expected to classify it into genre(s) listed in ['+ ",".join(class_list) + ']. Sample is provided below: '}]
    #Provide 3 samples for AI model reference and learning 
    for movie in movies:
        messages.append({
            'role': 'system',
            'content': movie['description'] + " should response: [" +','.join(movie['genres']) + ']'
        })
    #Provide the user input
    messages.append({'role': 'user', 'content': input_description})
    #print(messages)
    response = Generation.call(model="qwen2-7b-instruct",
                               messages=messages,
                               # Set the output format to message.
                               result_format='message')
    #Return the result based on the status code
    if response.status_code == HTTPStatus.OK:
        return(response['output']['choices'][0]['message']['content'])
    else:
        return('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    

app = Flask(__name__)

# Define an endpoint for movie genre classification
@app.route('/classify', methods=['POST'])
def classify_movie():
    data = request.get_json()
    description = data.get('description')
    genres = movie_classifier(description)
    return jsonify({"genres": genres})

if __name__ == '__main__':
    app.run(debug=False, port=5000)